## 이벤트 캘린더 생성기

이 노트북은 여러 CSV 파일에 나뉘어 저장된 웹사이트 페이지 방문 기록을 분석하여, 의미 있는 이벤트 기간을 추출하고, 그 결과를 `.ics` 파일 형식의 캘린더로 만드는 과정을 담고 있음.

**주요 작업 흐름:**
1. **데이터 로드 및 병합:** `all_pages_visit_daily_`로 시작하는 모든 CSV 파일을 불러와 하나의 데이터프레임으로 병합.
2. **데이터 정제:** 방문 수가 적은 데이터를 제거하고, 불필요한 페이지를 제외.
3. **연속된 이벤트 기간 추출:** 연속된 날짜에 방문 기록이 있는 페이지만을 필터링.
4. **이벤트 그룹화:** 연속된 날짜를 기준으로 각 이벤트를 그룹화.
5. **캠페인 정보 매핑:** 페이지 이름에 해당하는 캠페인 이름을 외부 엑셀 파일에서 불러와 매핑.
6. **캘린더 파일 생성:** 최종적으로 정리된 이벤트 정보를 바탕으로 `.ics` 캘린더 파일을 생성.

In [ ]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pandas as pd
import datetime
import xlwings as xw
import vobject

### 1. 데이터 로드 및 병합

In [ ]:
# 현재 폴더에서 'all_pages_visit_daily_'로 시작하는 CSV 파일 모두 읽어오기.
# 각 CSV 파일은 월별 방문 기록을 담고 있음.

path = os.getcwd()
csv_files = [f for f in os.listdir(path) if f.startswith('all_pages_visit_daily_') and f.endswith('.csv')]

# 여러 개의 데이터프레임을 담을 리스트 생성.
df_list = []
for file in csv_files:
    df = pd.read_csv(os.path.join(path, file), index_col=False, parse_dates=['Date'])
    df_list.append(df)

# 모든 월별 데이터프레임을 하나로 병합.
df_all_month = pd.concat(df_list, ignore_index=True)
df_all_month.head()

### 2. 데이터 정제

In [ ]:
# 분석 신뢰도를 높이기 위해, 일일 방문(Visits) 수가 특정 기준 이하인 데이터는 분석에서 제외.
VISITS_THRESHOLD = 1000

# 방문 수가 기준치보다 큰 데이터만 남김.
df_filtered = df_all_month[df_all_month['Visits'] > VISITS_THRESHOLD].copy()

# 결측치가 있는 행을 제거하고, 'Pages'와 'Date'를 기준으로 데이터 정렬.
df_filtered.dropna(how='any', inplace=True)
df_filtered.sort_values(by=['Pages', 'Date'], ascending=True, inplace=True)
df_filtered.reset_index(drop=True, inplace=True)

# 분석에 필요한 컬럼만 선택.
df_filtered = df_filtered[['Pages', 'Date', 'Visits']]

# 분석에서 제외할 페이지 목록.
del_page_names = ['sec:event:event-end', 'sec:event:indexexhibitioncollection']
df_filtered = df_filtered[~df_filtered['Pages'].isin(del_page_names)]

df_filtered.head()

### 3. 연속된 이벤트 기간 추출

In [ ]:
# 각 페이지별로 날짜가 연속되는지 확인하기 위해, 하루 전과 하루 후의 날짜 계산.
# pandas의 shift 기능을 사용하면 그룹별로 이전/이후 데이터를 효율적으로 가져오기 가능.
df_filtered['one_day_before'] = df_filtered.groupby('Pages')['Date'].shift(1)
df_filtered['one_day_after'] = df_filtered.groupby('Pages')['Date'].shift(-1)

# 현재 날짜와 하루 전/후 날짜의 차이 계산.
is_one_day_before_sequence = (df_filtered['Date'] - df_filtered['one_day_before']) == datetime.timedelta(days=1)
is_one_day_after_sequence = (df_filtered['one_day_after'] - df_filtered['Date']) == datetime.timedelta(days=1)

# 하루 전 또는 하루 후에 연속된 데이터가 있는 경우 'sequence' 컬럼을 True로 설정.
df_filtered['sequence'] = is_one_day_before_sequence | is_one_day_after_sequence

# 연속된 날짜를 가진 데이터만 필터링.
df_sequence_only = df_filtered[df_filtered['sequence'] == True].copy()
df_sequence_only.reset_index(drop=True, inplace=True)

df_sequence_only.head()

### 4. 이벤트 그룹화

In [ ]:
# 연속된 날짜들을 하나의 그룹으로 묶는 작업 수행.
# 날짜의 차이가 1일이 아닌 경우, 새로운 그룹이 시작된 것으로 간주.
df_sequence_only['date_diff'] = df_sequence_only.groupby('Pages')['Date'].diff()
df_sequence_only['new_group'] = df_sequence_only['date_diff'] != datetime.timedelta(days=1)

# 새로운 그룹이 시작될 때마다 그룹 번호를 1씩 증가시켜 고유한 그룹 ID 부여.
df_sequence_only['group_by_pages'] = df_sequence_only.groupby('Pages')['new_group'].cumsum()

df_sequence_only.head()

### 5. 캠페인 정보 매핑

In [ ]:
# 'page_campaign_name.xlsx' 엑셀 파일에서 페이지 이름과 캠페인 이름이 매핑된 데이터 불러오기.
# DRM 보안 프로그램으로 인해 xlwings를 사용하여 엑셀을 직접 실행하고 데이터 읽어오기.
try:
    workbook = xw.Book('page_campaign_name.xlsx')
    sheet = workbook.sheets[0]
    df_page_campaign_name = sheet.used_range.options(pd.DataFrame, header=1, index=False).value
    workbook.app.quit()

    # 페이지 이름을 key, 캠페인 이름을 value로 하는 딕셔너리 생성.
    campaign_dic = df_page_campaign_name.set_index('page_name')['campaign_name'].to_dict()

    # map 함수를 사용하여 캠페인 이름 매핑.
    df_sequence_only['campaign_name'] = df_sequence_only['Pages'].map(campaign_dic)
except Exception as e:
    print(f"엑셀 파일 처리 중 오류 발생: {e}")
    # 엑셀 파일이 없는 경우를 대비하여 빈 컬럼 생성.
    df_sequence_only['campaign_name'] = np.nan

# 캠페인 이름이 없는 경우, 페이지 이름으로 채우기.
mask = df_sequence_only['campaign_name'].isna() & (df_sequence_only['Pages'].str.startswith('sec:event:') | df_sequence_only['Pages'].str.startswith('sec:new-release:'))
df_sequence_only.loc[mask, 'campaign_name'] = df_sequence_only.loc[mask, 'Pages']

# 최종적으로 캠페인 이름이 없는 데이터는 제거.
df_campaign_only = df_sequence_only.dropna(subset=['campaign_name']).copy()
df_campaign_only['group_by_pages'] = df_campaign_only['group_by_pages'].astype(int)

df_campaign_only.head()

### 6. 캘린더 파일 생성

In [ ]:
# 각 캠페인 그룹별로 시작일과 종료일 계산.
df_campaign_schedule = df_campaign_only.groupby(['campaign_name', 'group_by_pages']).agg(
    start_date=('Date', 'min'),
    end_date=('Date', 'max')
).reset_index()

# 동일한 캠페인 이름에 여러 그룹이 있을 경우, (1/3), (2/3) 와 같이 표시하기 위해 그룹 수 계산.
df_campaign_schedule['group_max'] = df_campaign_schedule.groupby('campaign_name')['group_by_pages'].transform('max')

# 캘린더에 표시될 최종 캠페인 이름 생성.
df_campaign_schedule['campaign'] = df_campaign_schedule.apply(
    lambda x: f"{x['campaign_name']} ({x['group_by_pages']}/{x['group_max']})" if x['group_max'] > 1 else x['campaign_name'], axis=1
)

df_campaign_schedule = df_campaign_schedule[['campaign', 'start_date', 'end_date']]

df_campaign_schedule.head()

In [ ]:
# vobject 라이브러리를 사용하여 iCalendar(.ics) 파일 생성.
cal = vobject.iCalendar()

for index, row in df_campaign_schedule.iterrows():
    event = cal.add('vevent')
    event.add('summary').value = row['campaign']
    event.add('dtstart').value = row['start_date'].date()
    # dtend는 종료일 다음 날로 설정해야 해당일까지 포함.
    event.add('dtend').value = row['end_date'].date() + datetime.timedelta(days=1)

# 생성된 캘린더 객체를 'campaign_calander.ics' 파일로 저장.
try:
    with open('campaign_calander.ics', 'wb') as f:
        f.write(cal.serialize().encode('utf-8'))
    print("'campaign_calander.ics' 파일이 성공적으로 생성됨.")
except Exception as e:
    print(f"파일 저장 중 오류 발생: {e}")